# Using TensorBoard to view results when using TensorFlow

TensorBoard is a very fussy piece of software.  It was written in a Linux environment, and while it works in windows,  there are a number of issues related to how it
starts and stops operations.

It is a server that reads "log" files from various TensorFlow operations and shows you results in a browser.   It can run in a browser window, or as a plug-in within
a Jupyter notebook.  To get it to completely stop operating you have to:
    
    -Exit Jupyter notebook or Jupyter lab,  and stop the Jupyter server
    -Clear the log files and Tensorboard .info files 

https://www.tensorflow.org/tensorboard/get_started

This file is really just a test-file to see if we have TensorBoard running and to sort out issues related to tensorboard

In [1]:
import tensorflow as tf
import datetime
import os

# Manual deletion of old log files

Look in your current director for the directory "logs", or "logs_tb" and remove it, also check c:\logs and c:\logs_tb

You don't need to do this the first time you run Tensorboard

Delete the files before you start Tensorboard!

## Warning: Tensorflow leaves ".info" files behind that can interfere the next time you try to start it.  You need to clean these up

The code below deletes the tensorboard .info files,   you will need to replace hdavi with your username

In [2]:
# tensorboard does not shut down cleanly on it's own.  You have to manually delete a file so that it does not restart
# using the previous data set.   Shut it down (on Windows) using this command,   you need to change the user name from hdavi to your
# user name

dir = "C:\\Users\\hdavi\\AppData\\Local\\Temp\\.tensorboard-info"
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [3]:
# attempt to automatically delete old log files,  works if TensorBoard is not running anywhere

dir = "C:\\logs_tb"
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\logs_tb'

# Tensorflow model,  using the MNIST data set, as an example

This is the MNIST digits data set, which we have seen many times now.   The neural net is a pretty standard style construction, there is some dropout, and the layers have been named,  this helps later in viewing the graph of 
the network

In [15]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28),name="Flatten"),
    tf.keras.layers.Dense(512, activation='relu',name="Dense"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])


In [5]:
from tensorflow.keras.datasets.mnist import load_data

(x_train, y_train),(x_test, y_test) = load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Just checking the version of tensorflow being used here- I was using TensorFlow 2.6 which worked well,   I think I did have to install it form conda-forge

In [7]:
# checking behavior of Tensorflow
print(tf.__version__)

2.6.0


This is a check to see what devices are available for use in the calculation,   the CPU and the GPU

XLA stands for extended linear algebra,  whether this is available or not depends on the the version of the CUDA driver loaded on your machine.   It will speed operations in some cases

In [9]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

## Callbacks

TensorFlow has callbacks- these are functions specified in the model.fit, or model.predict member function calls that modify the behavior of the training process

Here we are adding a callback that will save the loss and the metrics to a log file on disk once per epoch.   The Tensorboard program can then access this log file to display the loss and other info 
for each step of the training process.    It is sort of like writing the history to disk

One other helpful callback we haven't seen is the early stopping callback, which stops the training at a minimum in the cross validation loss.   This is another way to avoid overfitting, by stopping the training early.

In [16]:
# create and train a basic model, kind of short on training epochs, just to save time


model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#These are are the steps used to set up a log file that Tensorplot will plot from once the model is fitted
# the directory is an "offset" from your current working directory.
# The callback is a step used in model fitting that writes to the log file

log_dir = "logs_tb/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2214 - accuracy: 0.9344 - val_loss: 0.1140 - val_accuracy: 0.9655
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0983 - accuracy: 0.9701 - val_loss: 0.0856 - val_accuracy: 0.9744
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0710 - accuracy: 0.9777 - val_loss: 0.0723 - val_accuracy: 0.9764
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0554 - accuracy: 0.9828 - val_loss: 0.0605 - val_accuracy: 0.9810
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0456 - accuracy: 0.9848 - val_loss: 0.0675 - val_accuracy: 0.9776


In [17]:
#Just to see what this model structure looks like and the number of parameters involved

model. summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
Dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Using TensorBoard to look at our results

TensorBoard can be run alone in a browser window, or we can load an extension into Jupyter lab/notebook, so that TensorBoard runs inside the Notebook.

TensorBoard does have it's own server, just as Jupyter notebooks do.  This is why TensorBoard doesn't always shut down correctly particularly on windows.

In [18]:
# load the tensorboard extension for jupyter notebooks,  by loading Tensorboard into Jupyter notebook and then starting it
%load_ext tensorboard

In [19]:
%tensorboard --logdir logs_tb/fit

Question/Action

The tensorboard plots show us the training process,  with the loss and accuracy (or other desired metric),  that is helpful and useful

It also shows us a graph that shows how the model and the training process is structured, this is the graph of the computations used.  Have a look at it.  It contains a lot of unfamilar terms,   we will cover these a bit later this 
semester.  YOu can download it as a .png file which makes it easier to look at,   do this and open it in a photo viewer.

In the graph view, if you click on the "Tag" option on the left side and select "keras", it will show you the Keras network structure.   Double click on this graph ("sequential"), it will show you more detail of the network


Question/Action

Add an early stopping callback to this model, and then increase the number of epochs.   (Google tensorflow callbacks).

What parameters are available in the early stopping callback to let you control the stopping point?

What other possibly useful callbacks are available?

### Watermarks

We may have already discussed this, but below are the watermarks for this file,  they indicate the version of Python used,  other info and then the versions of all the imported libraries.   Nice things to know about a project

Like TensorBoard above, this is an add-on extention to Jupyter notebook, not a Python function

In [ ]:
%load_ext watermark
%watermark

In [ ]:
%watermark --iversions